<a href="https://colab.research.google.com/github/OsyaginVictor/cost-of-cars/blob/main/cost_of_cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.


## Подготовка данных

Запишем признаки,чтобы удобно было искать нужные.

DateCrawled — дата скачивания анкеты из базы

VehicleType — тип автомобильного кузова

RegistrationYear — год регистрации автомобиля

Gearbox — тип коробки передач

Power — мощность (л. с.)

Model — модель автомобиля

Kilometer — пробег (км)

RegistrationMonth — месяц регистрации автомобиля

FuelType — тип топлива

Brand — марка автомобиля

Repaired — была машина в ремонте или нет

DateCreated — дата создания анкеты

NumberOfPictures — количество фотографий автомобиля

PostalCode — почтовый индекс владельца анкеты (пользователя)

LastSeen — дата последней активности пользователя

Целевой признак

Price — цена (евро)

In [ ]:
import pandas as pd
import numpy as np
import re
from scipy import stats as st
import matplotlib.pyplot as plt

from IPython.display import display
from numpy.random import RandomState
from scipy import stats

from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import OrdinalEncoder
import lightgbm
import catboost
%matplotlib inline


In [ ]:
df = pd.read_csv('/datasets/autos.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head(20)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


Приведем данные к змеиному языку.

In [ ]:
df.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', i).lower()for i in df.columns]
df.columns

Index(['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
       'power', 'model', 'kilometer', 'registration_month', 'fuel_type',
       'brand', 'repaired', 'date_created', 'number_of_pictures',
       'postal_code', 'last_seen'],
      dtype='object')

Посмотрим на наши данные

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   kilometer           354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  repaired            283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

In [ ]:
df.shape

(354369, 16)

In [ ]:
df.describe()

,price,registration_year,power,kilometer,registration_month,number_of_pictures,postal_code
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Отсмотрев данные,можно понять,что в столбцах 'date_crawled' , 'date_created' , 'last_seen' неверно указан тип данных - эти данные никак не влияют на стоимость автомобиля,их можно удалить.

В столбце 'number_of_pictures' везде проставлены нули,так что можно этот столбец удалить. Так же для нашей модели не важны данные столбца "postal_code", этот столбец тоже удалим.

В столбце с целевым признаком,есть нулевые значения,это явная ошибка эти данные нужно удалить.

Так же,в столбце 'Power' можно заметить выбросы и нули,строки с выбросами можно удалить. Строки в которых значения л\с меньше 40 можно удалить,так как это слишком маленькое значение для машины,не стал приводить данные,где присутствуют нули к среднему или медианному значению,так как очень большой разброс в мощности автомобилей. Здесь мы так же посмотрим на мощность автомобилей на рынке. На данный момент,машина с самым большим кол-вом л\с - это Dagger GT компании TranStar Racing LLC.Мощность автомобиля составляет 2028 л\с,но это машина скорее для энтузиастов,ведь на ней можно ехать всего лишь 10 минут,ведь расход топлива составляет 20л.в минуту.Поэтому не будем операться на эту машину,возьмем более реальную модель - Bugatti. Кол-во л\с варируется от 1200 до 1500.Возьмем максимальное значение от этих чисел 1500 и удалим значения больше этого числа.

Заметил,что в столбце 'Kilometer' есть много машин с пробегом в 150к,что делать с такими данными не знаю,поэтому оставим так как есть.


In [ ]:
df = df.drop(['number_of_pictures', 'date_created', 'date_crawled', 'last_seen','postal_code'], axis=1)

Посмотрим на кол-во дубликатов в данных

In [ ]:
df.duplicated().sum()

27543

Удалим эти дубликаты

In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df.loc[df['price'] != 0]

In [ ]:
df = df.loc[(df[['power']] < 1500).any(axis=1)]

In [ ]:
df = df.loc[(df[['power']] > 40).any(axis=1)]

In [ ]:
df.describe()

,price,registration_year,power,kilometer,registration_month
count,281750.000000,281750.000000,281750.000000,281750.000000,281750.000000
mean,4798.019507,2003.550037,121.105111,128624.223602,5.944220
std,4592.838452,32.311454,58.037598,36560.539254,3.604135
min,1.000000,1000.000000,41.000000,5000.000000,0.000000
25%,1300.000000,1999.000000,76.000000,125000.000000,3.000000
50%,3099.000000,2003.000000,110.000000,150000.000000,6.000000
75%,6900.000000,2008.000000,150.000000,150000.000000,9.000000
max,20000.000000,9999.000000,1436.000000,150000.000000,12.000000


In [ ]:
result1 = df[df['power']<100]

In [ ]:
result = df[df['power']>1000]

In [ ]:
result1

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
16,300,NaN,2016,NaN,60,polo,150000,0,petrol,volkswagen,NaN
17,1750,small,2004,auto,75,twingo,150000,2,petrol,renault,no
22,2900,NaN,2018,manual,90,meriva,150000,5,petrol,opel,no
...,...,...,...,...,...,...,...,...,...,...,...
354342,600,small,1998,manual,54,corsa,150000,1,petrol,opel,no
354345,1700,small,1999,manual,68,justy,70000,11,petrol,subaru,no
354356,999,convertible,2000,manual,95,megane,150000,4,petrol,renault,NaN
354357,1690,wagon,2004,manual,55,fabia,150000,4,petrol,skoda,NaN


In [ ]:
result

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired
1816,3200,small,2004,manual,1398,corolla,5000,6,petrol,toyota,no
5328,500,wagon,1999,manual,1001,astra,150000,7,petrol,opel,NaN
7720,1500,small,2000,manual,1400,NaN,150000,0,petrol,honda,NaN
19826,3390,sedan,2009,manual,1240,micra,60000,3,petrol,nissan,no
21609,200,small,2000,manual,1200,NaN,125000,4,petrol,lancia,NaN
...,...,...,...,...,...,...,...,...,...,...,...
342305,2350,bus,2002,manual,1250,galaxy,150000,0,gasoline,ford,NaN
345756,850,NaN,2005,manual,1003,ka,5000,12,petrol,ford,no
347016,450,sedan,1996,auto,1004,vectra,80000,5,petrol,opel,yes
348968,250,small,1999,manual,1241,ypsilon,150000,5,petrol,lancia,yes


Даже после удаления строк можно заметить что в данных много ошибок,взять например машину nissan micra sedan. Кол-во л\с реального автомобиля варируется от 50 до 117 л\с,в таблице же указаны 1240.Будь эти данные реальны,я бы попросил всё досканально проверить,но имеем,что имеем,поэтому будем продолжать строить модели для предсказания стоимости автомобиля.

In [ ]:
df

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes
...,...,...,...,...,...,...,...,...,...,...,...
354361,5250,NaN,2016,auto,150,159,150000,12,NaN,alfa_romeo,no
354362,3200,sedan,2004,manual,225,leon,150000,5,petrol,seat,yes
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


Так же посмотрим на количество пропусков в столбце с моделью машины

In [ ]:
f"Количество объявлений с незаполненной моделью: {len(df.loc[df['model'].isna()])}"

'Количество объявлений с незаполненной моделью: 11690'

Заполнить эти пропуски корректно не получится,а это один из основных  признаков стоимости автомобиля,чтобы модель нормально работала удалим эти пропуски

In [ ]:
df = df.loc[~df['model'].isna()]

в столбце repaired так же есть пропуски,их заполним заглушкой unknown

In [ ]:
df['repaired'] = df['repaired'].fillna('unknown')

/tmp/ipykernel_119/3541733968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['repaired'] = df['repaired'].fillna('unknown')


В столбце с годом регистрации автомобиля так же есть ошибки, чтобы модель лучше работала нужно удалить данные,где год регистрации больше чем 2023.

In [ ]:
df = df.loc[df['registration_year'] <= 2023]

Такие параметры как VehicleType, Gearbox, FuelType c большой вероятностью одинаковы в одинаковых моделях автомобилей. Заполним пропуски в данных признаках наиболее частым значением той же модели.

In [ ]:
df['vehicle_type'] = df['vehicle_type'].fillna(df
                                                  .groupby('model')['vehicle_type']
                                                  .transform(lambda x: x.value_counts().idxmax())
                                                 )

In [ ]:
df['gearbox'] = df['gearbox'].fillna(df
                                         .groupby('model')['vehicle_type']
                                         .transform(lambda x: x.value_counts().idxmax())
                                        )

In [ ]:
df['fuel_type'] = df['fuel_type'].fillna(df
                                           .groupby('model')['vehicle_type']
                                           .transform(lambda x: x.value_counts().idxmax())
                                          )

Посмотрим на данные после преобразования

In [ ]:
df

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes
6,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no
...,...,...,...,...,...,...,...,...,...,...,...
354361,5250,wagon,2016,auto,150,159,150000,12,wagon,alfa_romeo,no
354362,3200,sedan,2004,manual,225,leon,150000,5,petrol,seat,yes
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [ ]:
result2 = df[df['registration_month']<1]

In [ ]:
result2

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired
9,999,small,1998,manual,101,golf,150000,0,sedan,volkswagen,unknown
16,300,small,2016,small,60,polo,150000,0,petrol,volkswagen,unknown
36,1600,other,1991,manual,75,kadett,70000,0,sedan,opel,unknown
80,250,wagon,2000,manual,155,156,150000,0,petrol,alfa_romeo,yes
92,250,small,2000,small,60,ka,150000,0,small,ford,unknown
...,...,...,...,...,...,...,...,...,...,...,...
354204,250,wagon,1993,manual,75,golf,150000,0,petrol,volkswagen,no
354206,1700,sedan,1997,manual,101,a4,150000,0,petrol,audi,yes
354297,400,wagon,1996,manual,150,a4,150000,0,petrol,audi,unknown
354326,1300,small,1999,manual,75,2_reihe,125000,0,small,peugeot,unknown


In [ ]:
len(df['registration_month'].unique())

13

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270048 entries, 2 to 354368
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               270048 non-null  int64 
 1   vehicle_type        270048 non-null  object
 2   registration_year   270048 non-null  int64 
 3   gearbox             270048 non-null  object
 4   power               270048 non-null  int64 
 5   model               270048 non-null  object
 6   kilometer           270048 non-null  int64 
 7   registration_month  270048 non-null  int64 
 8   fuel_type           270048 non-null  object
 9   brand               270048 non-null  object
 10  repaired            270048 non-null  object
dtypes: int64(5), object(6)
memory usage: 24.7+ MB


Дальше произведем кодировку при помощи  one-hot encoding.Так же попробуем совместить признак Model c признаком Brand и закодировать значением: Model.size() / Brand.size()

In [ ]:
df['brand_model'] = df['brand'].astype(str) + ' ' + df['model'].astype(str)
df['model_code'] = df['brand_model'].map(df.groupby('brand_model').size()) / \
                     df['brand'].map(df.groupby('brand').size())
data = df.drop(['brand_model', 'brand', 'model'], axis=1)

In [ ]:
def get_dummies_func(df, column, prefix):
    df1 = pd.get_dummies(df[column], prefix=prefix, drop_first=True)
    df = df.drop(column, axis=1)
    return data.join(df1)

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'>`pd.get_dummies` - Хороший инструмент для быстрого анализа, но для новых данных если набор категорий будет отличаться, получить данные с такой-же размерностью будет нельзя. Для возможности работы модели с новыми данными нужен OneHotEncoder.</font>

<div class="alert alert-info">
    <b>Комментарий студента:</b> Очень хочу разобраться в Охе,буду рад,если дашь полезные статьи или видео про это)
</div>

In [ ]:
data = get_dummies_func(data, 'vehicle_type', 'vtype')

In [ ]:
data = get_dummies_func(data, 'gearbox', 'gear')

In [ ]:
data = get_dummies_func(data, 'fuel_type', 'fuel')

In [ ]:
data = get_dummies_func(data, 'repaired', 'repair')

In [ ]:
data1 = data.drop(['vehicle_type', 'gearbox', 'fuel_type', 'repaired'], axis=1)

In [ ]:

data.head(10)

,price,vehicle_type,registration_year,gearbox,power,kilometer,registration_month,fuel_type,repaired,model_code,vtype_convertible,vtype_coupe,vtype_other,vtype_sedan,vtype_small,vtype_suv,vtype_wagon,gear_bus,gear_convertible,gear_coupe,gear_manual,gear_sedan,gear_small,gear_suv,gear_wagon,fuel_cng,fuel_convertible,fuel_coupe,fuel_electric,fuel_gasoline,fuel_hybrid,fuel_lpg,fuel_other,fuel_petrol,fuel_sedan,fuel_small,fuel_suv,fuel_wagon,repair_unknown,repair_yes
2,9800,suv,2004,auto,163,125000,8,gasoline,unknown,0.383912,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1500,small,2001,manual,75,150000,6,petrol,no,0.403654,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,3600,small,2008,manual,69,90000,7,gasoline,no,0.388695,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,650,sedan,1995,manual,102,150000,10,petrol,yes,0.560487,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
6,2200,convertible,2004,manual,109,150000,8,petrol,no,0.487545,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,14500,bus,2014,manual,125,30000,8,petrol,unknown,0.031211,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
9,999,small,1998,manual,101,150000,0,sedan,unknown,0.403654,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
10,2000,sedan,2004,manual,105,150000,12,petrol,no,0.247534,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
11,2799,wagon,2005,manual,140,150000,12,gasoline,yes,0.138516,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
12,999,wagon,1995,manual,115,150000,11,petrol,unknown,0.138516,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


Выделим признаки, целевой признак. Разделим выборку на обучающую и тестовую в соотношении 1:4.


In [ ]:
X = data1.drop('price', axis=1)
y = data1['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
f"Размер обучающей выборки: {X_train.shape}"

'Размер обучающей выборки: (216038, 35)'

In [ ]:
f"Размер тестовой  выборки: {X_test.shape}"

'Размер тестовой  выборки: (54010, 35)'

In [ ]:
#Проведём масштабирование признаков к стандартному нормальному распределению с помощью метода StandardScaler библиотеки sklearn

In [ ]:
#ss = StandardScaler()
#ss.fit(X_train)
#columns = X_train.columns
#X_train = pd.DataFrame(ss.transform(X_train), columns = columns)
#X_test = pd.DataFrame(ss.transform(X_test), columns = columns)

Проведём масштабирование признаков  с помощью метода LabelEncoder()

In [ ]:


# Создание экземпляра OrdinalEncoder
encoder = OrdinalEncoder()

# Применение порядкового кодирования к каждому столбцу с категориальными данными
for column in X_train.columns:
    if X_train[column].dtype == 'object':
        X_train[column] = encoder.fit_transform(X_train[column].values.reshape(-1, 1))
        X_test[column] = encoder.transform(X_test[column].values.reshape(-1, 1))

In [ ]:
X_train.head(10)

,registration_year,power,kilometer,registration_month,model_code,vtype_convertible,vtype_coupe,vtype_other,vtype_sedan,vtype_small,vtype_suv,vtype_wagon,gear_bus,gear_convertible,gear_coupe,gear_manual,gear_sedan,gear_small,gear_suv,gear_wagon,fuel_cng,fuel_convertible,fuel_coupe,fuel_electric,fuel_gasoline,fuel_hybrid,fuel_lpg,fuel_other,fuel_petrol,fuel_sedan,fuel_small,fuel_suv,fuel_wagon,repair_unknown,repair_yes
198829,2001,58,150000,2,0.271371,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
267536,2017,75,125000,7,0.403654,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
899,2003,135,5000,12,0.086485,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
146413,1985,72,150000,6,0.095780,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
172374,2003,231,150000,11,0.392252,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
291773,1981,200,125000,9,0.040114,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
347269,1991,45,150000,7,0.172276,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
209273,2001,125,150000,4,0.291377,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
151120,1993,60,150000,12,0.011369,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
46964,2006,166,80000,3,0.243634,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


Вывод

Данные были успешно загружены и проанализированы. Типы данных соответствуют описанию, дубликаты удалены.
Отобраны признаки, необходимые для построения моделей.

Категориальные признаки с небольшой ординальностью преобразованы с помощью one-hot encoding. Признак Model закодирован по формуле Model.size() / Brand.size()


Данные разделены на тестовую и обучающую выборки в соотношении 1:4, признаки масштабированы к стандартному нормальному распределению.

## Обучение моделей

Для начала создадим константную base-line модель. Она всегда будет предсказывать среднее значение Price. (среднее значение на практике дало меньшее значение RMSE, чем медианное). Рассчитаем RMSE для base-line модели.

In [ ]:
preds_base = pd.Series(data=y.mean(), index=y.index, dtype='int64')
rmse_base = mean_squared_error(y, preds_base) ** 0.5
f"RMSE для константной модели: {round(rmse_base, 2)}"

'RMSE для константной модели: 4602.0'

Сначала обучим линейную модель. Посчитаем скорость обучения, скорость предсказания и RMSE модели. Обучение будем проводить с использованием кросс-валидации на 5 фолдов.

In [ ]:
%%time
lm = LinearRegression()
mse = cross_val_score(lm, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

CPU times: user 2.56 s, sys: 1.58 s, total: 4.13 s
Wall time: 4.08 s


In [ ]:
f"RMSE для линейной модели на валидационной выборке: {round((-mse.mean()) ** 0.5, 2)}"

'RMSE для линейной модели на валидационной выборке: 3010.86'

In [ ]:
%%time
lm = LinearRegression()
lm.fit(X_train, y_train)

CPU times: user 603 ms, sys: 243 ms, total: 845 ms
Wall time: 807 ms


LinearRegression()

In [ ]:
%%time
preds_lm = lm.predict(X_test)

CPU times: user 12.2 ms, sys: 0 ns, total: 12.2 ms
Wall time: 7.17 ms


In [ ]:
mse_lm = mean_squared_error(y_test, preds_lm)
f"RMSE для линейной модели на тестовой выборке: {round((mse_lm) ** 0.5, 2)}"

'RMSE для линейной модели на тестовой выборке: 3052.07'

Lasso/Ridge

Попробуем обучить линейные модели с регуляризацией L1 и L2.

Lasso (L1 регуляризация)

In [ ]:
%%time
alphas = np.arange(1, 40, 4)
for alpha in alphas :
    lasso = Lasso(alpha = alpha)
    mse = cross_val_score(lasso, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    print("RMSE для Lasso alpha={:.2f} модели на валидационной выборке: {:.2f}".format(alpha, (-mse.mean()) ** 0.5))

RMSE для Lasso alpha=1.00 модели на валидационной выборке: 3011.25
RMSE для Lasso alpha=5.00 модели на валидационной выборке: 3015.26
RMSE для Lasso alpha=9.00 модели на валидационной выборке: 3017.39
RMSE для Lasso alpha=13.00 модели на валидационной выборке: 3020.25
RMSE для Lasso alpha=17.00 модели на валидационной выборке: 3024.00
RMSE для Lasso alpha=21.00 модели на валидационной выборке: 3027.93
RMSE для Lasso alpha=25.00 модели на валидационной выборке: 3031.67
RMSE для Lasso alpha=29.00 модели на валидационной выборке: 3035.81
RMSE для Lasso alpha=33.00 модели на валидационной выборке: 3040.41
RMSE для Lasso alpha=37.00 модели на валидационной выборке: 3044.91
CPU times: user 53.3 s, sys: 24.8 s, total: 1min 18s
Wall time: 1min 18s


In [ ]:
%%time
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

CPU times: user 19.9 s, sys: 5.73 s, total: 25.7 s
Wall time: 25.7 s


Lasso(alpha=0.1)

In [ ]:
%%time
preds_ls = lasso.predict(X_test)

CPU times: user 27.9 ms, sys: 36.8 ms, total: 64.7 ms
Wall time: 95.9 ms


In [ ]:
mse_ls = mean_squared_error(y_test, preds_ls)
f"RMSE для Lasso модели на тестовой выборке: {round((mse_ls) ** 0.5, 2)}"

'RMSE для Lasso модели на тестовой выборке: 3051.9'

Lasso не дало выигрыш в метрике, скорость обучения и предсказания немного выше, чем у Линейной модели без регуляризации.

Ridge (L2 регуляризация)

In [ ]:
%%time
alphas = np.arange(1, 40, 4)
for alpha in alphas :
    ridge = Ridge(alpha = alpha)
    mse = cross_val_score(ridge, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    print("RMSE для Ridge alpha={:.2f} модели на валидационной выборке: {:.2f}".format(alpha, (-mse.mean()) ** 0.5))

RMSE для Ridge alpha=1.00 модели на валидационной выборке: 3010.84
RMSE для Ridge alpha=5.00 модели на валидационной выборке: 3010.77
RMSE для Ridge alpha=9.00 модели на валидационной выборке: 3010.72
RMSE для Ridge alpha=13.00 модели на валидационной выборке: 3010.68
RMSE для Ridge alpha=17.00 модели на валидационной выборке: 3010.65
RMSE для Ridge alpha=21.00 модели на валидационной выборке: 3010.63
RMSE для Ridge alpha=25.00 модели на валидационной выборке: 3010.62
RMSE для Ridge alpha=29.00 модели на валидационной выборке: 3010.61
RMSE для Ridge alpha=33.00 модели на валидационной выборке: 3010.60
RMSE для Ridge alpha=37.00 модели на валидационной выборке: 3010.60
CPU times: user 7.97 s, sys: 6.89 s, total: 14.9 s
Wall time: 14.9 s


In [ ]:
%%time
ridge = Ridge(alpha=0.5)
ridge.fit(X_train, y_train)

CPU times: user 92 ms, sys: 60.3 ms, total: 152 ms
Wall time: 183 ms


Ridge(alpha=0.5)

In [ ]:
%%time
preds_rg = ridge.predict(X_test)

CPU times: user 21.6 ms, sys: 18.4 ms, total: 40 ms
Wall time: 91.4 ms


In [ ]:
mse_rg = mean_squared_error(y_test, preds_rg)
f"RMSE для Ridge модели на тестовой выборке: {round((mse_rg) ** 0.5, 2)}"

'RMSE для Ridge модели на тестовой выборке: 3052.07'

Ridge не дал выигрыш в метрике, но скорость обучения выше, чем у Линейной модели без регуляризации:

Ridge fit time: 234 ms

LinearRegression fit time: 1.1 s

LightGBM

Попробуем обучить модель с градиентным бустингом: LightGBM.
    
Подбор параметров проведём с помощью GridSearch библиотеки sklearn.

In [ ]:
numerical_features = ['price', 'registration_year', 'power', 'kilometer']
categorical_features = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'repaired', 'brand ']

In [ ]:
X = data1.drop('price', axis=1)
y = data1['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
f"Размер обучающей выборки: {X_train.shape}"

'Размер обучающей выборки: (216038, 35)'

In [ ]:
f"Размер обучающей выборки: {X_test.shape}"

'Размер обучающей выборки: (54010, 35)'

In [ ]:
categorical_features = [col for col in list(data.columns) if col not in numerical_features]

In [ ]:
X_train.info()
categorical_features

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216038 entries, 198829 to 153496
Data columns (total 35 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   registration_year   216038 non-null  int64  
 1   power               216038 non-null  int64  
 2   kilometer           216038 non-null  int64  
 3   registration_month  216038 non-null  int64  
 4   model_code          216038 non-null  float64
 5   vtype_convertible   216038 non-null  uint8  
 6   vtype_coupe         216038 non-null  uint8  
 7   vtype_other         216038 non-null  uint8  
 8   vtype_sedan         216038 non-null  uint8  
 9   vtype_small         216038 non-null  uint8  
 10  vtype_suv           216038 non-null  uint8  
 11  vtype_wagon         216038 non-null  uint8  
 12  gear_bus            216038 non-null  uint8  
 13  gear_convertible    216038 non-null  uint8  
 14  gear_coupe          216038 non-null  uint8  
 15  gear_manual         216038 no

['vehicle_type',
 'gearbox',
 'registration_month',
 'fuel_type',
 'repaired',
 'model_code',
 'vtype_convertible',
 'vtype_coupe',
 'vtype_other',
 'vtype_sedan',
 'vtype_small',
 'vtype_suv',
 'vtype_wagon',
 'gear_bus',
 'gear_convertible',
 'gear_coupe',
 'gear_manual',
 'gear_sedan',
 'gear_small',
 'gear_suv',
 'gear_wagon',
 'fuel_cng',
 'fuel_convertible',
 'fuel_coupe',
 'fuel_electric',
 'fuel_gasoline',
 'fuel_hybrid',
 'fuel_lpg',
 'fuel_other',
 'fuel_petrol',
 'fuel_sedan',
 'fuel_small',
 'fuel_suv',
 'fuel_wagon',
 'repair_unknown',
 'repair_yes']

In [ ]:
%%time
lgbm = lightgbm.LGBMRegressor(n_jobs=6)
lgbm.fit(X_train, y_train)

CPU times: user 4.14 s, sys: 24.5 ms, total: 4.17 s
Wall time: 4.12 s


LGBMRegressor(n_jobs=6)

In [ ]:
%%time
train_data = lightgbm.Dataset(X_train, label=y_train,
                              free_raw_data=False,
                              categorical_feature=categorical_features)
test_data = lightgbm.Dataset(X_test, label=y_test)

param_grid = {'learning_rate': [0.3, 0.5],
              'max_depth': [15, 25],
             }

lgbm = lightgbm.LGBMRegressor(n_jobs = 6)

# инициализируем GridSearchCV
grid_search = GridSearchCV(estimator = lgbm,
                           param_grid = param_grid,
                           cv = 2,
                           n_jobs = -1,
                           verbose = 0,
                           scoring = 'neg_mean_squared_error',
                          )
grid_search.fit(X_train, y_train)

CPU times: user 24.6 s, sys: 385 ms, total: 25 s
Wall time: 25 s


GridSearchCV(cv=2, estimator=LGBMRegressor(n_jobs=6), n_jobs=-1,
             param_grid={'learning_rate': [0.3, 0.5], 'max_depth': [15, 25]},
             scoring='neg_mean_squared_error')

<font color='blue'><b>Комментарий ревьюера 2: </b></font>\
<font color='blue'>Время обучения модели с лучшими параметрами на всех данных (в конце подбора) можно вытащить как  `grid_search.refit_time_`</font>

In [ ]:
print(grid_search.best_params_)

{'learning_rate': 0.3, 'max_depth': 15}


In [ ]:
print('RMSE для LightGBM модели на валидационной выборке: {:.2f}'.format((-grid_search.best_score_) ** 0.5))

RMSE для LightGBM модели на валидационной выборке: 1662.73


In [ ]:
%%time
preds_lgbm = grid_search.best_estimator_.predict(X_test)

CPU times: user 363 ms, sys: 49 µs, total: 364 ms
Wall time: 309 ms


In [ ]:
mse_lgbm = mean_squared_error(y_test, preds_lgbm)
f"RMSE для LightGBM модели на тестовой выборке: {round((mse_lgbm) ** 0.5, 2)}"

'RMSE для LightGBM модели на тестовой выборке: 1646.06'

Проведено обучение моделей линейной регрессии без/с регуляризацией, а также модели градиентного бустинга LightGBM.

Измерены время обучения/предсказания моделей, а так же качество предсказаний по метрике RMSE.

## Анализ моделей

Все обученные модели смогли улучшить RMSE базовой модели.

Линейная модель без регуляризации для данной задачи показала качество предсказаний не хуже, чем Lasso и Ridge. Метрика RMSE для LightGBM в два раза ниже:

Base-line: 4602

LinearModel: 3010.86

Lasso 3011.25

Ridge 3010.84

LightGBM: 1662.73

По времени обучения Ridge модель оказалась быстрее всех. Lasso значительно уступает линейной модели без регуляризации. Обучение LightGBM оказалось не намного дольше:

LinearModel: 802 ms

Lasso 26.9 s

Ridge 188 ms

LightGBM: 4.12 s
По времени предсказания линейные модели ведут себя одинаково. LightGBM более тяжелая модель, время предсказания выше на порядок.


Вывод

Исходя из условий задачи, заказчику важны: 1. качество предсказания; 2. скорость предсказания; 3. время обучения.

Исходя из того, что порядок криттериев также учитывает их приоритет, следует отдать предпочтение LightGBM, т.к. она имеет лучшее качество предсказания. Линейная модель выигрывает в скорости предсказания. А Ridge получилась самая быстрая модель в обучении.
